In [1]:
import local_nav as lv
import math as m

In [2]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
aw(node.lock())

Node 5a4009fd-ceaf-4496-b738-e8cf662f77f5

In [3]:
def prox_less_threshold(prox_val : list, threshold: int): 
    for i in prox_val : 
        if i >= threshold : 
            return False
    return True

# FOR SOME REASON pyhton doesn't have a sign function, so I'm building it...
def sign (a) : 
    if a >= 0 : 
        return 1
    return -1

In [4]:
WHEEL_RADIUS = 2
WHEEL_BASE = 10
REJ_WEIGHT = 50
WAIT_TIME = 0.2
def vect_dot_prod(a : tuple, b : tuple) : 
    prod = 0
    for i,j in zip(a, b) : 
        prod +=  i*j
    return prod


In [5]:
SPEED_100 = 6.5 #cm/s
await node.wait_for_variables()
prox_read = list(node["prox.horizontal"])
v = {"motor.right.target" : [100], "motor.left.target" : [100]}
max_iter = 5000
counter = 0
await node.set_variables(v)
while ((counter < max_iter) and prox_less_threshold(prox_read, 1500)): 
    await client.sleep(0.2)
    await node.wait_for_variables()
    prox_read = list(node["prox.horizontal"])
    #print(prox_read)
    counter = counter + 1 
# for i in range(10): 
#     await client.sleep(0.2)
#     counter = counter + 1
#     print(counter*100*0.2)
curr_pos = (counter*SPEED_100*WAIT_TIME, 0)
curr_dir = (100, 0)
objectif = (curr_pos[0]+60, 0)
second_counter = 0
obstacle_not_passed = True
#print(prox_less_threshold(prox_read, 1000))
while (abs(curr_pos[0]-objectif[0]) > 2 or abs(curr_pos[1]-objectif[1])> 2):
    prox_read = list(node["prox.horizontal"])
    if (prox_less_threshold(prox_read, 1000)) and obstacle_not_passed:
        print("PASSED")
        v["motor.left.target"] = [100]
        v["motor.right.target"] = [100]
        await node.set_variables(v)
        await client.sleep(2)
        obstacle_not_passed = False
        curr_pos = (curr_pos[0]+SPEED_100*2*curr_dir[0]/curr_dir_norm, curr_pos[1]+SPEED_100*2*curr_dir[1]/curr_dir_norm)
        continue
    vect = lv.vect_calculation(objectif, (curr_pos),curr_dir,  prox_read[:5], REJ_WEIGHT, debug=True)
    norm_vect = m.sqrt(vect[0]**2 + vect[1]**2)
    curr_dir_norm = m.sqrt(curr_dir[0]**2+curr_dir[1]**2)
    angle_command = m.atan2(vect[1], vect[0])
    delta_speed = sign(angle_command)*min(abs(angle_command/WAIT_TIME*800/m.pi), 300)
    left_speed = int(100+(delta_speed/2))
    right_speed = int(100-(delta_speed/2))
    v["motor.left.target"] = [left_speed]
    v["motor.right.target"] = [right_speed]
    await node.set_variables(v)
    second_counter = second_counter +1
    #print(f"{second_counter} : curr_pos = ({curr_pos[0]:.2f}, {curr_pos[1]:.2f}), vect = {angle_command:.2f} and delta_speed = {delta_speed:.2f}, curr_dir = ({curr_dir[0]:.2f}, {curr_dir[1]:.2f})")
    await client.sleep(WAIT_TIME)
    #print(curr_dir, delta_speed*WAIT_TIME*m.pi/100, m.cos(delta_speed*WAIT_TIME*m.pi/100))
    omega = delta_speed*m.pi/800
    curr_dir = (curr_dir[0]*m.cos((omega)*WAIT_TIME)-curr_dir[1]*m.sin((omega)*WAIT_TIME), curr_dir[0]*m.sin((omega)*WAIT_TIME)+curr_dir[1]*m.cos((omega)*WAIT_TIME))
    curr_dir_norm = m.sqrt(curr_dir[0]**2+curr_dir[1]**2)
    await node.wait_for_variables()
    curr_pos = (curr_pos[0]+ WAIT_TIME*SPEED_100*curr_dir[0]/curr_dir_norm, curr_pos[1] + curr_dir[1]*SPEED_100*WAIT_TIME/curr_dir_norm)
    

v["motor.left.target"] = [100]
v["motor.right.target"] = [100] 
await node.set_variables(v)
await client.sleep(0.2)

v["motor.left.target"] = [0]
v["motor.right.target"] = [0] 
await node.set_variables(v)

angle to abjectif :  0.0
rejection : (0.15446907415336678, 0.06398318548153985)
attraction : (60.0, 0.0)
angle to abjectif :  0.06241863478628211
rejection : (0.17501033008359287, 0.07249165227601626)
attraction : (58.58816334235289, 3.6617499035856085)
angle to abjectif :  0.06301470745349406
rejection : (0.26425793977844175, 0.1490362876289427)
attraction : (57.29107735963898, 3.6149665807095994)
angle to abjectif :  0.15274385935950838
rejection : (0.2894049934520982, 0.16743156542726229)
attraction : (55.46655090848637, 8.538683010929653)
angle to abjectif :  0.1518461925114753
rejection : (0.30498229615480266, 0.20128501695453496)
attraction : (54.20379161684456, 8.294486982905612)
angle to abjectif :  0.20094980133232196
rejection : (0.4271186102322009, 0.33465387417441905)
attraction : (52.483045677763066, 10.690746870317236)
angle to abjectif :  0.39600996972262553
rejection : (0.4633727268588123, 0.4633727268588122)
attraction : (48.31067945271734, 20.198591217863033)
PASSED
a

In [31]:
v["motor.left.target"] = [0]
v["motor.right.target"] = [0]
await node.set_variables(v)

In [72]:
aw(node.stop())
aw(node.unlock())

In [15]:
node.stop()
node.unlock()

<generator object ClientAsyncNode.unlock at 0x7f81657f01e0>

In [18]:
a = (1,2)
b = (1, -1)
print(vect_dot_prod(a,b))

1 1
2 -1
-1


In [5]:
for i in range(100) : 
    await node.wait_for_variables(["motor.left.target", "motor.right.target"])
    prox_reead_db = list(node["prox.horizontal"])[:5]
    print(prox_reead_db)
    print(lv.vect_calculation((60,0),(0,0),(100,0),prox_reead_db, REJ_WEIGHT, debug=True))
    await client.sleep(5)

[0, 2618, 3708, 3110, 2117]
angle to abjectif :  0.0
rejection : (2.3468442025087373, 0.05921173399899779)
attraction : (60.0, 0.0)
(13.063115949825253, -1.184234679979956)
[1728, 4241, 4375, 4238, 2934]
angle to abjectif :  0.0
rejection : (14.440269694508753, -0.38887857766647926)
attraction : (60.0, 0.0)
(-228.80539389017508, 7.777571553329585)
[1728, 4241, 4373, 4240, 2934]
angle to abjectif :  0.0
rejection : (14.302164171873898, -0.3833668890195785)
attraction : (60.0, 0.0)
(-226.04328343747795, 7.66733778039157)
[0, 2838, 4361, 4355, 4302]
angle to abjectif :  0.0
rejection : (13.26368352791513, 2.0064795466246075)
attraction : (60.0, 0.0)
(-205.27367055830263, -40.12959093249215)
[0, 2059, 4359, 4388, 4372]
angle to abjectif :  0.0
rejection : (14.383393125095642, 2.9082577664238904)
attraction : (60.0, 0.0)
(-227.66786250191285, -58.16515532847781)
[0, 0, 2865, 2540, 1992]
angle to abjectif :  0.0
rejection : (1.0691570344010122, 0.16791389904991216)
attraction : (60.0, 0.0)
(

CancelledError: 

In [33]:
v["motor.left.target"] = [0]
v["motor.right.target"] = [100]
await node.set_variables(v)
await client.sleep(1)
v["motor.left.target"] = [0]
v["motor.right.target"] = [0]
await node.set_variables(v)